In [ ]:
import io
import pathlib
import requests
import zipfile

import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)

In [ ]:
ROOT_DIR = pathlib.Path("..")

DATASETS_URL = "https://github.com/KAUST-Academy/python-for-data-analysis/raw/november-2022/datasets.zip"
EXAMPLES_URL = "https://github.com/KAUST-Academy/python-for-data-analysis/raw/november-2022/examples.zip"

response = requests.get(DATASETS_URL)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall(ROOT_DIR)


response = requests.get(EXAMPLES_URL)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall(ROOT_DIR)


In [ ]:
DATASETS_DIR = ROOT_DIR / "datasets"
EXAMPLES_DIR = ROOT_DIR / "examples"


In [ ]:
!ls $DATASETS_DIR

In [ ]:
!ls $EXAMPLES_DIR

In [ ]:
!cat $EXAMPLES_DIR/ex1.csv

In [ ]:
df = pd.read_csv(EXAMPLES_DIR / "ex1.csv")
df

In [ ]:
!cat $EXAMPLES_DIR/ex2.csv

In [ ]:
pd.read_csv(EXAMPLES_DIR / "ex2.csv", header=None)

In [ ]:
pd.read_csv(EXAMPLES_DIR / "ex2.csv", names=["a", "b", "c", "d", "message"])

In [ ]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv(EXAMPLES_DIR / "ex2.csv", names=names, index_col="message")

In [ ]:
!cat $EXAMPLES_DIR/csv_mindex.csv

In [ ]:
parsed = pd.read_csv(EXAMPLES_DIR / "csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

In [ ]:
!cat $EXAMPLES_DIR/ex3.txt

In [ ]:
result = pd.read_csv(EXAMPLES_DIR / "ex3.txt", sep="\s+")
result

In [ ]:
!cat $EXAMPLES_DIR/ex4.csv

In [ ]:
pd.read_csv(EXAMPLES_DIR / "ex4.csv", skiprows=[0, 2, 3])

In [ ]:
!cat $EXAMPLES_DIR/ex5.csv

In [ ]:
result = pd.read_csv(EXAMPLES_DIR / "ex5.csv")
result

In [ ]:
pd.isna(result)

In [ ]:
result = pd.read_csv(EXAMPLES_DIR / "ex5.csv", na_values=["NULL"])
result

In [ ]:
result2 = pd.read_csv(EXAMPLES_DIR / "ex5.csv", keep_default_na=False)
result2

In [ ]:
result2.isna()

In [ ]:
result3 = pd.read_csv(EXAMPLES_DIR / "ex5.csv", keep_default_na=False,
                      na_values=["NA"])
result3

In [ ]:
result3.isna()

In [ ]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv(EXAMPLES_DIR / "ex5.csv", na_values=sentinels,
            keep_default_na=False)

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
result = pd.read_csv(EXAMPLES_DIR / "ex6.csv")
result

In [ ]:
pd.read_csv(EXAMPLES_DIR / "ex6.csv", nrows=5)

In [ ]:
chunker = pd.read_csv(EXAMPLES_DIR / "ex6.csv", chunksize=1000)
type(chunker)

In [ ]:
chunker = pd.read_csv(EXAMPLES_DIR / "ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [ ]:
tot[:10]

In [ ]:
data = pd.read_csv(EXAMPLES_DIR / "ex5.csv")
data

In [ ]:
data.to_csv(EXAMPLES_DIR / "out.csv")

In [ ]:
!cat $EXAMPLES_DIR/out.csv

In [ ]:
import sys

data.to_csv(sys.stdout, sep="|")

In [ ]:
data.to_csv(sys.stdout, na_rep="NULL")

In [ ]:
data.to_csv(sys.stdout, index=False, header=False)

In [ ]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

In [ ]:
!cat $EXAMPLES_DIR/ex7.csv

In [ ]:
import csv

f = open(EXAMPLES_DIR / "ex7.csv")
reader = csv.reader(f)

In [ ]:
for line in reader:
    print(line)
f.close()

In [ ]:
with open(EXAMPLES_DIR / "ex7.csv") as f:
    lines = list(csv.reader(f))

In [ ]:
header, values = lines[0], lines[1:]

In [ ]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

In [ ]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [ ]:
import json

result = json.loads(obj)
result

In [ ]:
asjson = json.dumps(result)
asjson

In [ ]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

In [ ]:
!cat $EXAMPLES_DIR/example.json

In [ ]:
data = pd.read_json(EXAMPLES_DIR / "example.json")
data

In [ ]:
data.to_json(sys.stdout)
data.to_json(sys.stdout, orient="records")

In [ ]:
tables = pd.read_html(EXAMPLES_DIR / "fdic_failed_bank_list.html")
print(len(tables))
failures = tables[0]
failures.head()

In [ ]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

In [ ]:
from lxml import objectify

path = DATASETS_DIR / "mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()

In [ ]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)
perf.head()

In [ ]:
perf2 = pd.read_xml(path)
perf2.head()

In [ ]:
frame = pd.read_csv(EXAMPLES_DIR / "ex1.csv")
print(frame)
frame.to_pickle(EXAMPLES_DIR / "frame_pickle")

In [ ]:
pd.read_pickle(EXAMPLES_DIR / "frame_pickle")

In [ ]:
!rm $EXAMPLES_DIR/frame_pickle

In [ ]:
fec = pd.read_parquet(DATASETS_DIR / 'fec/fec.parquet')

In [ ]:
xlsx = pd.ExcelFile(EXAMPLES_DIR / "ex1.xlsx")

In [ ]:
xlsx.sheet_names

In [ ]:
xlsx.parse(sheet_name="Sheet1")

In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

In [ ]:
frame = pd.read_excel(EXAMPLES_DIR / "ex1.xlsx", sheet_name="Sheet1")
frame

In [ ]:
writer = pd.ExcelWriter(EXAMPLES_DIR / "ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.save()

In [ ]:
frame.to_excel(EXAMPLES_DIR / "ex2.xlsx")

In [ ]:
!rm $EXAMPLES_DIR/ex2.xlsx

In [ ]:
!rm -f $EXAMPLES_DIR/mydata.h5

In [ ]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})
store = pd.HDFStore(EXAMPLES_DIR / "mydata.h5")
store["obj1"] = frame
store["obj1_col"] = frame["a"]
store

In [ ]:
store["obj1"]

In [ ]:
store.put("obj2", frame, format="table")
store.select("obj2", where=["index >= 10 and index <= 15"])
store.close()

In [ ]:
frame.to_hdf(EXAMPLES_DIR / "mydata.h5", "obj3", format="table")
pd.read_hdf(EXAMPLES_DIR / "mydata.h5", "obj3", where=["index < 5"])

In [ ]:
import os

os.remove(EXAMPLES_DIR / "mydata.h5")

In [ ]:
import requests

url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

In [ ]:
data = resp.json()
data[0]["title"]

In [ ]:
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

In [ ]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)
con.commit()

In [ ]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

In [ ]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
import sqlalchemy as sqla

db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

In [ ]:
!rm mydata.sqlite